# Exercise 5

Using the files *transition_probabilities.csv* and *ovservation_likelihoods.csv*, compute the likelihood of POS tags for the following word sequence:

    I want to race

## Task 1

Load the given files and:

a) Write a function to calculate all possible tag sequences for the given word sentence.

In [95]:
import pandas as pd
import itertools

df_obs = pd.read_csv("observation_likelihoods.csv", ",", index_col=0)
df_obs = df_obs.rename(columns=lambda x : x.strip())

df_trans = pd.read_csv("transisition_probabilities.csv", ",", index_col=0)
df_trans = df_trans.rename(columns=lambda x : x.strip())
sentence = "I want to race"

def possible_tag_seq(probability_df_obs, sentence):
    return list(itertools.combinations_with_replacement(list(probability_df_obs.index), len(sentence.split())))

print(possible_tag_seq(df_obs, sentence))


[('VB', 'VB', 'VB', 'VB'), ('VB', 'VB', 'VB', 'TO'), ('VB', 'VB', 'VB', 'NN'), ('VB', 'VB', 'VB', 'PPSS'), ('VB', 'VB', 'TO', 'TO'), ('VB', 'VB', 'TO', 'NN'), ('VB', 'VB', 'TO', 'PPSS'), ('VB', 'VB', 'NN', 'NN'), ('VB', 'VB', 'NN', 'PPSS'), ('VB', 'VB', 'PPSS', 'PPSS'), ('VB', 'TO', 'TO', 'TO'), ('VB', 'TO', 'TO', 'NN'), ('VB', 'TO', 'TO', 'PPSS'), ('VB', 'TO', 'NN', 'NN'), ('VB', 'TO', 'NN', 'PPSS'), ('VB', 'TO', 'PPSS', 'PPSS'), ('VB', 'NN', 'NN', 'NN'), ('VB', 'NN', 'NN', 'PPSS'), ('VB', 'NN', 'PPSS', 'PPSS'), ('VB', 'PPSS', 'PPSS', 'PPSS'), ('TO', 'TO', 'TO', 'TO'), ('TO', 'TO', 'TO', 'NN'), ('TO', 'TO', 'TO', 'PPSS'), ('TO', 'TO', 'NN', 'NN'), ('TO', 'TO', 'NN', 'PPSS'), ('TO', 'TO', 'PPSS', 'PPSS'), ('TO', 'NN', 'NN', 'NN'), ('TO', 'NN', 'NN', 'PPSS'), ('TO', 'NN', 'PPSS', 'PPSS'), ('TO', 'PPSS', 'PPSS', 'PPSS'), ('NN', 'NN', 'NN', 'NN'), ('NN', 'NN', 'NN', 'PPSS'), ('NN', 'NN', 'PPSS', 'PPSS'), ('NN', 'PPSS', 'PPSS', 'PPSS'), ('PPSS', 'PPSS', 'PPSS', 'PPSS')]


b) Write a function to select the most probable tag sequence.

In [90]:
def most_prob(dataframe, sentence):
    seq = []
    tokens = sentence.split()
    for token in tokens:
        seq.append(dataframe.nlargest(1, token).index[0])
    return seq
display(df.columns)
print(most_prob(df, sentence))

Index(['I', 'want', 'to', 'race'], dtype='object')

['PPSS', 'VB', 'TO', 'NN']


c) Discuss your solution.

The function possible_tag_seq recieves a sentence and a data farme with all possible tags and their probabilities. As the probabilites are not relevant for the task, the function calculates the number of tokens in the sentence by using the split funciton and counting (this only works if the resulting tokens match those of the file). The function then takes all possible POs-Tag  retrieved from the DataFrame and using Itertools calcualtes all possible permutaitons of these POS-Tags with a length equal to the number of tokens.

The function most_prob simply looks at which POS-Tag is most probable using the passed dataframe and the pandas function nlargest for each token and chooses that one. By doing so, the most probable sequence is determined.

## Task 2

Load the given files and follow the algorithm shown in slide 13 of our Lab Session 5 to:

a) Compute the initialization step of Viterbi.

b) Compute the recursion step of Viterbi.

c) Compute termination step of Viterbi.